In [1]:
import re
import numpy as np
import requests
import math
import time
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
import json
from tqdm import tqdm

init_notebook_mode(connected=True)
PI = math.pi

In [8]:
'''
* 根据经纬度信息计算dist，提供了四个不同的计算函数
* 计算函数参考：https://www.cnblogs.com/osnosn/p/14505778.html
* Vincenty和Haversine两种计算方式，Vincenty的精度更高
'''
def vincenty(lat1,lon1,lat2,lon2):
    '''
    计算地球上两点之间的大圆距离的Haversine公式。它是一个地理空间中使用的测地线距离（最短路径）的公式，通常用于计算两个经纬度点之间的距离。
    Haversine公式考虑了地球的曲率，因此比简单的平面距离计算要准确。这个公式使用球面三角学，假设地球是一个完美的球体，并给出了从一个点到另一个点的弧长。
    '''
    a=6378137.0       #vincentyConstantA(WGS84) ##单位:米
    b=6356752.3142451 #vincentyConstantB(WGS84) ##单位:米
    f=1/298.257223563 #vincentyConstantF(WGS84)
    L = math.radians(lon2 - lon1)
    U1 = math.atan((1 - f) *math.tan(math.radians(lat1)))
    U2 = math.atan((1 - f) *math.tan(math.radians(lat2)))
    sinU1 =math.sin(U1)
    cosU1 =math.cos(U1)
    sinU2 =math.sin(U2)
    cosU2 =math.cos(U2)
    lambda1 = L
    lambdaP = 2 * math.pi
    iterLimit = 20

    sinLambda = 0.0
    cosLambda = 0.0
    sinSigma = 0.0
    cosSigma = 0.0
    sigma = 0.0
    alpha = 0.0
    cosSqAlpha = 0.0
    cos2SigmaM = 0.0
    C = 0.0
    while (abs(lambda1 - lambdaP) > 1e-12 and --iterLimit > 0) :
        sinLambda =math.sin(lambda1)
        cosLambda =math.cos(lambda1)
        sinSigma = math.sqrt((cosU2 * sinLambda) * (cosU2 * sinLambda) + (cosU1 * sinU2 - sinU1 * cosU2 * cosLambda) * (cosU1 * sinU2 - sinU1 * cosU2 * cosLambda))
        if (sinSigma == 0) :
            return 0
        cosSigma = sinU1 * sinU2 + cosU1 * cosU2 * cosLambda
        sigma = math.atan2(sinSigma, cosSigma)
        alpha = math.asin(cosU1 * cosU2 * sinLambda / sinSigma)
        cosSqAlpha = math.cos(alpha) * math.cos(alpha)
        cos2SigmaM = cosSigma - 2 * sinU1 * sinU2 / cosSqAlpha
        C = f / 16 * cosSqAlpha * (4 + f * (4 - 3 * cosSqAlpha))
        lambdaP = lambda1
        lambda1 = L + (1 - C) * f * math.sin(alpha)* (sigma + C * sinSigma * (cos2SigmaM + C * cosSigma * (-1 + 2 * cos2SigmaM * cos2SigmaM)))

    if iterLimit == 0 :
        return 0.0

    uSq = cosSqAlpha * (a * a - b * b) / (b * b)
    A = 1 + uSq / 16384 * (4096 + uSq * (-768 + uSq * (320 - 175 * uSq)))
    B = uSq / 1024 * (256 + uSq * (-128 + uSq * (74 - 47 * uSq)))
    deltaSigma = B * sinSigma * (cos2SigmaM + B / 4 * (cosSigma * (-1 + 2 * cos2SigmaM * cos2SigmaM) - B / 6 * cos2SigmaM * (-3 + 4 * sinSigma * sinSigma) * (-3 + 4 * cos2SigmaM * cos2SigmaM)))
    s = b * A * (sigma - deltaSigma)
    d = s
    return d

def haversine(lat1,lng1,lat2,lng2):
    '''
    使用Haversine公式计算地球上两点之间距离的Python函数。这个函数接受四个参数：两个点的纬度（lat1, lat2）和经度（lng1, lng2）。函数的返回值是两点之间的距离，单位是米，并且结果四舍五入到十分之一米。
    Haversine公式是一个用于计算球面上两点之间最短距离的公式，假设球体是一个完美的圆球。在这个函数中，地球的半径被设置为6378.137千米，这是地球的平均半径。
    '''
    RR = 6378.137 #为地球赤道半径，单位:千米
    lng1, lat1, lng2, lat2 = map(math.radians, [float(lng1), float(lat1), float(lng2), float(lat2)]) # 经纬度转换成弧度
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    distance=2*math.asin(math.sqrt(a))*RR*1000
    distance=round(distance,1)
    
    return distance

def LL2Dist(Lat1,Lng1,Lat2,Lng2):
    '''
    用于计算地球上两点之间距离的Python函数，这个函数考虑了地球的椭球形状。这个算法是一个修正的Haversine公式，它使用地球的赤道半径和极半径来更准确地计算椭球体上的距离。
    函数LL2Dist接受四个参数：两个点的纬度（Lat1, Lat2）和经度（Lng1, Lng2）。函数的返回值是两点之间的距离，单位是米。
    '''
    ra = 6378137.0        # radius of equator: meter
    rb = 6356752.3142451  # radius of polar: meter
    flatten = (ra - rb) / ra  # Partial rate of the earth
    if Lat1==Lat2 and Lng1==Lng2:
        return 0
    # change angle to radians
    radLatA = math.radians(Lat1)
    radLonA = math.radians(Lng1)
    radLatB = math.radians(Lat2)
    radLonB = math.radians(Lng2)

    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB))**2 / math.cos(x / 2)**2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB))**2 / math.sin(x / 2)**2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    
    return distance

def FlatternDist(lat1,lng1,lat2,lng2):
    '''
    用于计算地球上两点之间距离的Python函数，这个函数也考虑了地球的椭球形状。这个算法是一个简化的Vincenty公式，它是一种更精确的地理距离计算方法，尤其是在计算长距离时。
    函数FlatternDist接受四个参数：两个点的纬度（lat1, lat2）和经度（lng1, lng2）。函数的返回值是两点之间的距离，单位是米。
    '''
    ra = 6378137.0        # radius of equator: meter
    #rb = 6356752.3142451  # radius of polar: meter
    flatten=1/298.257223563 #vincentyConstantF(WGS84)
    #flatten = (ra - rb) / ra  # Partial rate of the earth
    if lat1==lat2 and lng1==lng2:
        return 0
    # change angle to radians
    f = math.radians((lat1+lat2)/2)
    g = math.radians((lat1-lat2)/2)
    l = math.radians((lng1-lng2)/2)
    sf = math.sin(f)
    sg = math.sin(g)
    sl = math.sin(l)
    sg = sg * sg
    sl = sl * sl
    sf = sf * sf

    s = sg * (1 - sl)+(1 - sf) * sl
    c = (1 - sg) * (1 - sl) + sf * sl

    w = math.atan(math.sqrt(s / c))
    r = math.sqrt(s * c) / w
    d = 2 * w * ra
    h1 = (3 * r - 1) / 2 / c
    h2 = (3 * r + 1) / 2 / s

    return d * (1 + flatten * (h1 * sf * (1 - sg) - h2 * (1 - sf) * sg))

In [3]:
'''
* 对百度地图API提供的经纬度格式转化，最终转为WGS-84
* 坐标信息说明：https://lbsyun.baidu.com/skins/MySkin/resources/iframs/coordinate.html?update
'''
def _transformlat(coordinates):
    lng = coordinates[ : , 0] - 105
    lat = coordinates[ : , 1] - 35
    ret = -100 + 2 * lng + 3 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * np.sqrt(np.fabs(lng))
    ret += (20 * np.sin(6 * lng * PI) + 20 *
            np.sin(2 * lng * PI)) * 2 / 3
    ret += (20 * np.sin(lat * PI) + 40 *
            np.sin(lat / 3 * PI)) * 2 / 3
    ret += (160 * np.sin(lat / 12 * PI) + 320 *
            np.sin(lat * PI / 30.0)) * 2 / 3
    return ret

def _transformlng(coordinates):
    lng = coordinates[ : , 0] - 105
    lat = coordinates[ : , 1] - 35
    ret = 300 + lng + 2 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * np.sqrt(np.fabs(lng))
    ret += (20 * np.sin(6 * lng * PI) + 20 *
            np.sin(2 * lng * PI)) * 2 / 3
    ret += (20 * np.sin(lng * PI) + 40 *
            np.sin(lng / 3 * PI)) * 2 / 3
    ret += (150 * np.sin(lng / 12 * PI) + 300 *
            np.sin(lng / 30 * PI)) * 2 / 3
    return ret

def gcj02_to_wgs84(coordinates):
    """
    GCJ-02转WGS-84
    :param coordinates: GCJ-02坐标系的经度和纬度的numpy数组
    :returns: WGS-84坐标系的经度和纬度的numpy数组
    """
    ee = 0.006693421622965943  # 偏心率平方
    a = 6378245  # 长半轴
    lng = coordinates[ : , 0]
    lat = coordinates[ : , 1]
    is_in_china = (lng > 73.66) & (lng < 135.05) & (lat > 3.86) & (lat < 53.55)
    _transform = coordinates[is_in_china]  #只对不在国内的坐标做偏移
    
    dlat = _transformlat(_transform)
    dlng = _transformlng(_transform)
    radlat = _transform[ : , 1] / 180 * PI
    magic = np.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = np.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * PI)
    dlng = (dlng * 180.0) / (a / sqrtmagic * np.cos(radlat) * PI)
    mglat = _transform[ : , 1] + dlat
    mglng = _transform[ : , 0] + dlng
    coordinates[is_in_china] = np.array([_transform[ : , 0] * 2 - mglng, _transform[ : , 1] * 2 - mglat]).T
    return coordinates

def bd09_to_gcj02(coordinates):
    """
    BD-09转GCJ-02
    :param coordinates: BD-09坐标系的经度和纬度的numpy数组
    :returns: GCJ-02坐标系的经度和纬度的numpy数组
    """
    x_pi = PI * 3000 / 180
    x = coordinates[ : , 0] - 0.0065
    y = coordinates[ : , 1] - 0.006
    z = np.sqrt(x * x + y * y) - 0.00002 * np.sin(y * x_pi)
    theta = np.arctan2(y, x) - 0.000003 * np.cos(x * x_pi)
    lng = z * np.cos(theta)
    lat = z * np.sin(theta)
    coordinates = np.array([lng, lat]).T
    return coordinates

def bd09_to_wgs84(coordinates):
    """
    BD-09转WGS-84
    :param coordinates: BD-09坐标系的经度和纬度的numpy数组
    :returns: WGS-84坐标系的经度和纬度的numpy数组
    """
    return gcj02_to_wgs84(bd09_to_gcj02(coordinates))

def mercator_to_bd09(mercator):
    """
    墨卡托转BD-09
    :param coordinates: GCJ-02坐标系的经度和纬度的numpy数组
    :returns: WGS-84坐标系的经度和纬度的numpy数组
    """
    MCBAND = [12890594.86, 8362377.87, 5591021, 3481989.83, 1678043.12, 0]
    MC2LL = [[1.410526172116255e-08,   8.98305509648872e-06,    -1.9939833816331,        200.9824383106796,       -187.2403703815547,
                 91.6087516669843,        -23.38765649603339,      2.57121317296198,        -0.03801003308653,       17337981.2],
            [-7.435856389565537e-09,  8.983055097726239e-06,   -0.78625201886289,       96.32687599759846,       -1.85204757529826,
                 -59.36935905485877,      47.40033549296737,       -16.50741931063887,      2.28786674699375,        10260144.86],
            [-3.030883460898826e-08,  8.98305509983578e-06,    0.30071316287616,        59.74293618442277,       7.357984074871,
                 -25.38371002664745,      13.45380521110908,       -3.29883767235584,       0.32710905363475,        6856817.37],
            [-1.981981304930552e-08,  8.983055099779535e-06,   0.03278182852591,        40.31678527705744,       0.65659298677277,
                 -4.44255534477492,       0.85341911805263,        0.12923347998204,        -0.04625736007561,       4482777.06], 
            [3.09191371068437e-09,    8.983055096812155e-06,   6.995724062e-05,         23.10934304144901,       -0.00023663490511,
                 -0.6321817810242,        -0.00663494467273,       0.03430082397953,        -0.00466043876332,       2555164.4],  
            [2.890871144776878e-09,   8.983055095805407e-06,   -3.068298e-08,           7.47137025468032,        -3.53937994e-06,
                 -0.02145144861037,       -1.234426596e-05,        0.00010322952773,        -3.23890364e-06,         826088.5]]   
    
    x = np.abs(mercator[ : , 0])
    y = np.abs(mercator[ : , 1])
    coef = np.array([MC2LL[index] for index in (np.tile(y.reshape((-1, 1)), (1, 6)) < MCBAND).sum(axis=1)])   
    return converter(x, y, coef)

def converter(x, y, coef):
    x_temp = coef[ : ,0] + coef[ : ,1] * np.abs(x)
    x_n = np.abs(y) / coef[ : ,9]
    y_temp = coef[ : ,2] + coef[ : ,3] * x_n + coef[ : ,4] * x_n ** 2 + coef[ : ,5] * x_n ** 3 + coef[ : ,6] * x_n ** 4 + coef[ : ,7] * x_n ** 5 + coef[ : ,8] * x_n ** 6
    x[x < 0] = -1
    x[x >= 0] = 1
    y[y < 0] = -1
    y[y >= 0] = 1    
    x_temp *= x
    y_temp *= y
    coordinates = np.array([x_temp, y_temp]).T
    
    return coordinates



def format_lng_lat(data1, data2):
    '''
    将数据转为{"(0,1)":"lng":{},"lat":{}}格式
    '''
    result = {}
    
    for i in range(len(data2) - 1):
        start_node = data2[i]
        end_node = data2[i + 1]
        lngs = []
        lats = []
        
        for point in data1:
            if (point[0] >= start_node[0] and point[0] <= end_node[0]) or \
               (point[0] <= start_node[0] and point[0] >= end_node[0]):
                if (point[1] >= min(start_node[1], end_node[1]) and point[1] <= max(start_node[1], end_node[1])) or \
                   (point[1] <= min(start_node[1], end_node[1]) and point[1] >= max(start_node[1], end_node[1])):
                    lngs.append(point[0])
                    lats.append(point[1])
                    
        result[f"({i},{i+1})"] = {"lng": lngs, "lat": lats}
    
    return result

In [11]:
# 以可视化形式运行
if __name__ == '__main__':
    mapbox_access_token ="pk.eyJ1IjoibHVrYXNtYXJ0aW5lbGxpIiwiYSI6ImNpem85dmhwazAyajIyd284dGxhN2VxYnYifQ.HQCmyhEXZUTz3S98FMrVAQ"
    layout = go.Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat= 23.12864583, #广州市纬度
                lon= 113.2648325 #广州市经度
                #lat= 22.547, #深圳维度
                #lon= 114.085947 #深圳经度
            ),
            pitch=0,
            zoom=10
        ),
    )

    null = None
    city_code = 257 #广州的城市编号
    #city_code = 340 #深圳的城市编号，城市编号表：https://blog.csdn.net/weixin_40191474/article/details/105686552
    station_info = requests.get('http://map.baidu.com/?qt=bsi&c=%s&t=%s' % (
                        city_code, 
                        int(time.time() * 1000)
                   )
    )
    station_info_json = eval(station_info.content)
    
    data = []
    marked = set()
    for railway in station_info_json['content']:
        uid = railway['line_uid']
        if uid in marked: #由于线路包括了来回两个方向，排除已绘制线路的反向线路
            continue

        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
        railway_json = requests.get('https://map.baidu.com/?qt=bsl&tps=&newmap=1&uid=%s&c=%s' % (uid, city_code), headers=headers)
        railway_json =  json.loads(railway_json.content) 
        
        trace_mercator = np.array(
            # 取出线路信息字典，以“|”划分后，取出第三部分信息，去掉末尾的“;”,获取墨卡托坐标序列
            railway_json['content'][0]['geo'].split('|')[2][ : -1].split(','), 
            dtype=float
        ).reshape((-1, 2)) 
        
        trace_coordinates = bd09_to_wgs84(mercator_to_bd09(trace_mercator))
            
        plots = [] #站台墨卡托坐标
        plots_name = [] #站台名称
        for plot in railway['stops']:
            plots.append([plot['x'], plot['y']])
            plots_name.append(plot['name'])
        plot_mercator = np.array(plots)
        plot_coordinates = bd09_to_wgs84(mercator_to_bd09(plot_mercator)) #站台经纬度
       
        color = railway_json['content'][0]['line_color'] #利用json所给线路的颜色 # 425492
        
        data.extend([
            # 地铁路线
            go.Scattermapbox(
                lon=trace_coordinates[:, 0], #路线点经度
                lat=trace_coordinates[:, 1], #路线点纬度
                mode='lines',
                # 设置路线的参数
                line=go.scattermapbox.Line(
                    width=2,
                    color=color
                ),         
                name=railway['line_name'], #线路名称，显示在图例（legend）上
                legendgroup=railway['line_name']
            ),
            
            # 地铁站台
            go.Scattermapbox(
                lon=plot_coordinates[:, 0], #站台经度
                lat=plot_coordinates[:, 1], #站台纬度
                mode='markers',   
                text=plots_name,
                # 设置标记点的参数
                marker=go.scattermapbox.Marker(
                    size=10,
                    color=color
                ),
                name=railway['line_name'], #线路名称，显示在图例（legend）及鼠标悬浮在标记点时的路线名上
                legendgroup=railway['line_name'], #设置与路线同组，当隐藏该路线时隐藏标记点
                showlegend=False #不显示图例（legend)
            )
        ])
            
        marked.add(uid) #添加已绘制线路的uid
        marked.add(railway['pair_line_uid']) #添加已绘制线路反向线路的uid

    fig = dict(data=data, layout=layout)
    #py.iplot(fig) #直接显示地图
    py.plot(fig, filename='Guangzhou_railway.html') #生成html文件并打开

In [15]:
# 以数据组织形式运行
if __name__ == '__main__':
    null = None
    city_code = 257 #广州的城市编号
    #city_code = 340 #深圳的城市编号
    station_info = requests.get('http://map.baidu.com/?qt=bsi&c=%s&t=%s' % (
                        city_code, 
                        int(time.time() * 1000)
                   )
    )
    
    station_info_json = eval(station_info.content)

    #with open('shenzhen_station_info.json', 'w', encoding="utf-8") as outfile:
    #    json.dump(station_info_json, outfile, ensure_ascii=False, indent=4)

    marked = set()
    wrs_line_name = []
    total_data = {}
    metro = []
    for railway in tqdm(station_info_json['content']):
        uid = railway['line_uid']
        if uid in marked: #由于线路包括了来回两个方向，需要排除已绘制线路的反向线路
            continue

        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
        railway_json = requests.get('https://map.baidu.com/?qt=bsl&tps=&newmap=1&uid=%s&c=%s' % (uid, city_code), headers=headers)
        
        railway_json =  json.loads(railway_json.content) #将json字符串转为python对象

        #with open('shenzhen_railway_info.json', 'w', encoding="utf-8") as outfile1:
        #    json.dump(railway_json, outfile1, ensure_ascii=False, indent=4)
        
        trace_mercator = np.array(
            # 取出线路信息字典，以“|”划分后，取出第三部分信息，去掉末尾的“;”,获取墨卡托坐标序列
            railway_json['content'][0]['geo'].split('|')[2][ : -1].split(','), 
            dtype=float
        ).reshape((-1, 2)) 
        
        trace_coordinates = bd09_to_wgs84(mercator_to_bd09(trace_mercator))
            
        plots = [] #站台墨卡托坐标
        plots_name = [] #站台名称
        for plot in railway['stops']:
            plots.append([plot['x'], plot['y']])
            plots_name.append(plot['name'])
        
        plot_mercator = np.array(plots)
        plot_coordinates = bd09_to_wgs84(mercator_to_bd09(plot_mercator)) #站台经纬度

        print(railway['line_name'])
        print(plots_name)

        # 无法很好处理数字线路和纯文字线路
        # metro_num = re.findall(r'\d+', railway['line_name'])
        # if metro_num in metro:
        #     lname = "Line"+str(re.findall(r'\d+', railway['line_name'])[0])+"-1"
        #     metro.append(metro_num)
        # if metro_num not in metro:
        #     try:
        #         lname = "Line"+str(re.findall(r'\d+', railway['line_name'])[0])
        #         metro.append(metro_num)
        #     except:
        #         lname = railway['line_name']
        lname = railway['line_name']
        wrs_line_name.append(lname)

        # Line information
        wrs_line_format = format_lng_lat(trace_coordinates, plot_coordinates)
        # cal dist
        for key_val in range(len(wrs_line_format)):
            dist = []
            allkey = wrs_line_format[list(wrs_line_format.keys())[key_val]]
            for pi in range(len(allkey['lng'])):
                distance = vincenty(allkey['lat'][0], allkey['lng'][0], allkey['lat'][pi], allkey['lng'][pi])
                dist.append(distance)
            allkey['dist']=dist
            wrs_line_format[str(list(wrs_line_format.keys())[key_val])] = allkey
        
        # id2name
        wrs_id2name = {str(i): value for i, value in enumerate(plots_name)}
        # name2id
        wrs_name2id = {value: int(i) for i, value in enumerate(plots_name)}
        # center_gps
        wrs_center_gps = {str(index): item.tolist() for index, item in enumerate(plot_coordinates)}
        # station_num
        wrs_station_num = len(plots_name)

        wrs_line_format["id2name"] = wrs_id2name,
        wrs_line_format["name2id"] = wrs_name2id,
        wrs_line_format["center_gps"] = wrs_center_gps,
        wrs_line_format["station_num"] = wrs_station_num

        total_data[lname] = wrs_line_format

        # 去除反方向的路线
        marked.add(uid)
        marked.add(railway['pair_line_uid'])
    
    total_data['Line_name'] = wrs_line_name
    with open('./guangzhou_Metro.json', 'w', encoding='utf-8') as file:
        json.dump(total_data, file, ensure_ascii=False, indent=4)

  2%|█▊                                                                                 | 1/46 [00:00<00:17,  2.53it/s]

广清城际线(花都-清城)
['花都', '乐同', '狮岭', '银盏', '龙塘镇', '清城']


  7%|█████▍                                                                             | 3/46 [00:00<00:10,  4.11it/s]

黄埔有轨电车1号线(地铁香雪-新丰路)
['地铁香雪', '区少年宫(萝岗)', '线坑', '市民广场', '会议中心', '地铁水西', '峻泰路', '水西(有轨)', '北师大实验学校', '地铁长平', '羌洞', '长岭居小学', '岭头', '干部健康中心', '岭头东', '贤江西', '贤江公园', '贤江', '新丰路']


  9%|███████▏                                                                           | 4/46 [00:01<00:12,  3.40it/s]

...

广州东环城际线(花都-白云机场北)
['花都', '花城街', '花山镇', '白云机场北']


 74%|████████████████████████████████████████████████████████████▌                     | 34/46 [00:08<00:01,  7.71it/s]

广佛线(新城东-沥滘)
['新城东', '东平', '世纪莲', '澜石', '魁奇路', '季华园', '同济路', '祖庙', '普君北路', '朝安', '桂城', '南桂路', '礌岗', '千灯湖', '金融高新区', '龙溪', '菊树', '西塱', '鹤洞', '沙涌', '沙园', '燕岗', '石溪', '南洲', '沥滘']


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:08<00:00,  5.30it/s]

地铁3号线北延段(机场北(2号航站楼)-体育西路)
['机场北(2号航站楼)', '机场南(1号航站楼)', '高增', '人和', '龙归', '嘉禾望岗', '白云大道北', '永泰', '同和', '京溪南方医院', '梅花园', '燕塘', '广州东站', '林和西', '体育西路']


In [ ]:
# 对四种不同的距离计算函数进行测试
# CDMetro中距离为1463.509

In [7]:
vincenty(30.710662, 104.089843, 30.701631, 104.080489)

1343.6459010525114

In [22]:
haversine(30.710662, 104.089843, 30.701631, 104.080489)

1346.2

In [24]:
LL2Dist(30.710662, 104.089843, 30.701631, 104.080489)

1343.6440681528352

In [26]:
FlatternDist(30.710662, 104.089843, 30.701631, 104.080489)

1343.645748651074